In [1]:
# !pip install kagglehub[pandas-datasets]

In [3]:
import pandas as pd
import os
from tqdm.auto import tqdm
tqdm.pandas()

import kagglehub
from kagglehub import KaggleDatasetAdapter

### Load Data

In [4]:
def load_data(data_id = "dbdmobile/myanimelist-dataset", anime_filename = "final_animedataset.csv", user_filename = "users-details-2023.csv"):

  anime_df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    data_id,
    anime_filename
  )

  user_df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    data_id,
    user_filename
  )

  return anime_df, user_df

In [5]:
anime_df, user_df = load_data()

/tmp/ipykernel_459067/1641063760.py:3: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  anime_df = kagglehub.load_dataset(
/tmp/ipykernel_459067/1641063760.py:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  user_df = kagglehub.load_dataset(


In [6]:
display(anime_df.head(3))  

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre
0,karthiga,21,9,2255153,Female,One Piece,TV,Manga,8.54,423868,91.0,35,"Action, Adventure, Comedy, Super Power, Drama,..."
1,karthiga,59,7,2255153,Female,Chobits,TV,Manga,7.53,175388,1546.0,188,"Sci-Fi, Comedy, Drama, Romance, Ecchi, Seinen"
2,karthiga,74,7,2255153,Female,Gakuen Alice,TV,Manga,7.77,33244,941.0,1291,"Comedy, School, Shoujo, Super Power"


In [7]:
display(user_df.head(3))

,Mal ID,Username,Gender,Birthday,Location,Joined,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched
0,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,2004-11-05T00:00:00+00:00,142.3,7.37,1.0,233.0,8.0,93.0,64.0,399.0,60.0,8458.0
1,3,Aokaado,Male,NaN,"Oslo, Norway",2004-11-11T00:00:00+00:00,68.6,7.34,23.0,137.0,99.0,44.0,40.0,343.0,15.0,4072.0
2,4,Crystal,Female,NaN,"Melbourne, Australia",2004-11-13T00:00:00+00:00,212.8,6.68,16.0,636.0,303.0,0.0,45.0,1000.0,10.0,12781.0


In [8]:
anime_df.columns

Index(['username', 'anime_id', 'my_score', 'user_id', 'gender', 'title',
       'type', 'source', 'score', 'scored_by', 'rank', 'popularity', 'genre'],
      dtype='object')

In [9]:
user_df.columns

Index(['Mal ID', 'Username', 'Gender', 'Birthday', 'Location', 'Joined',
       'Days Watched', 'Mean Score', 'Watching', 'Completed', 'On Hold',
       'Dropped', 'Plan to Watch', 'Total Entries', 'Rewatched',
       'Episodes Watched'],
      dtype='object')

### Join Final Anime and User Data

In [10]:
df = pd.merge(anime_df, user_df, left_on='user_id', right_on='Mal ID', how='inner')

In [11]:
df = df.drop("Mal ID", axis=1)

In [12]:
df.columns

Index(['username', 'anime_id', 'my_score', 'user_id', 'gender', 'title',
       'type', 'source', 'score', 'scored_by', 'rank', 'popularity', 'genre',
       'Username', 'Gender', 'Birthday', 'Location', 'Joined', 'Days Watched',
       'Mean Score', 'Watching', 'Completed', 'On Hold', 'Dropped',
       'Plan to Watch', 'Total Entries', 'Rewatched', 'Episodes Watched'],
      dtype='object')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16963197 entries, 0 to 16963196
Data columns (total 28 columns):
 #   Column            Dtype  
---  ------            -----  
 0   username          object 
 1   anime_id          int64  
 2   my_score          int64  
 3   user_id           int64  
 4   gender            object 
 5   title             object 
 6   type              object 
 7   source            object 
 8   score             float64
 9   scored_by         int64  
 10  rank              float64
 11  popularity        int64  
 12  genre             object 
 13  Username          object 
 14  Gender            object 
 15  Birthday          object 
 16  Location          object 
 17  Joined            object 
 18  Days Watched      float64
 19  Mean Score        float64
 20  Watching          float64
 21  Completed         float64
 22  On Hold           float64
 23  Dropped           float64
 24  Plan to Watch     float64
 25  Total Entries     float64
 26  Rewatched   

In [14]:
for col in df.columns:
    if df[col].isnull().sum()>0:
        print(f"{col} --> {df[col].isnull().sum()}")

rank --> 390522
genre --> 859
Gender --> 226699
Birthday --> 1985852
Location --> 464017


In [5]:
print(df.shape)
df = df.dropna()
print(df.shape)

(16963197, 28)
(14422331, 28)


Save merged data as intermediate results

In [ ]:
# out_dir = "./data"
# os.makedirs(out_dir, exist_ok=True)
# df.to_csv(out_dir + os.sep + "updated_anime_data.csv",index=False)

### Calculate Age

In [ ]:
out_dir = "./data"
df = pd.read_csv(out_dir + os.sep + "updated_anime_data.csv")

In [3]:
from datetime import datetime
def calculate_age(date_str):
    dt = datetime.fromisoformat(str(date_str))
    age = 2023 - dt.year
    return age

In [6]:
df["Age"] = df["Birthday"].progress_apply(calculate_age)

  0%|          | 0/14422331 [00:00<?, ?it/s]

In [7]:
df.head(3)

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,...,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched,Age
0,Damonashu,21,10,37326,Male,One Piece,TV,Manga,8.54,423868,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32
1,Damonashu,232,0,37326,Male,Cardcaptor Sakura,TV,Manga,8.21,121898,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32
2,Damonashu,249,4,37326,Male,InuYasha,TV,Manga,7.90,181978,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32


In [8]:
df.describe()

,anime_id,my_score,user_id,score,scored_by,rank,popularity,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched,Age
count,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07,1.442233e+07
mean,8.312846e+03,4.866148e+00,3.089743e+05,7.518487e+00,1.011596e+05,2.117444e+03,1.396107e+03,1.543867e+02,7.497146e+00,3.161089e+01,5.508759e+02,2.805567e+01,3.335600e+01,2.018251e+02,8.457240e+02,2.995948e+01,9.826659e+03,3.306857e+01
std,9.691629e+03,3.886869e+00,3.095009e+05,6.969011e-01,1.420862e+05,1.957535e+03,1.534479e+03,1.581398e+02,1.360829e+00,9.109186e+01,6.910277e+02,1.124916e+02,9.328616e+01,5.665929e+02,1.095285e+03,1.556110e+02,2.193307e+04,7.325124e+00
min,1.000000e+00,0.000000e+00,1.000000e+00,1.900000e+00,2.000000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.700000e+01
25%,9.080000e+02,0.000000e+00,8.633400e+04,7.110000e+00,1.569600e+04,5.600000e+02,2.990000e+02,6.510000e+01,7.090000e+00,5.000000e+00,1.730000e+02,2.000000e+00,2.000000e+00,2.000000e+01,2.900000e+02,0.000000e+00,3.911000e+03,3.000000e+01
50%,4.437000e+03,7.000000e+00,2.235020e+05,7.540000e+00,4.730300e+04,1.543000e+03,8.710000e+02,1.166000e+02,7.630000e+00,1.200000e+01,3.510000e+02,1.000000e+01,1.000000e+01,6.400000e+01,5.540000e+02,3.000000e+00,7.055000e+03,3.200000e+01
75%,1.169900e+04,8.000000e+00,4.104960e+05,8.000000e+00,1.218980e+05,3.158000e+03,1.941000e+03,1.978000e+02,8.160000e+00,2.800000e+01,6.840000e+02,2.700000e+01,3.200000e+01,1.720000e+02,1.026000e+03,2.400000e+01,1.208300e+04,3.500000e+01
max,3.786300e+04,1.000000e+01,1.291057e+06,9.250000e+00,1.009477e+06,9.527000e+03,1.026500e+04,9.010700e+03,1.000000e+01,2.934000e+03,1.164300e+04,4.493000e+03,2.522000e+03,8.863000e+03,2.054500e+04,1.321500e+04,1.778924e+06,7.020000e+02


In [9]:
df = df.drop( ["Birthday","Joined"] , axis=1)

Save intermediate results

In [ ]:
# out_dir = "./data/intermediate_results"
# os.makedirs(out_dir, exist_ok=True)
# df.to_csv(out_dir + os.sep + "cleaned_anime_data.csv",index=False)

### Check Validity of Location values

In [ ]:
out_dir = "./data/intermediate_results"
df = pd.read_csv(out_dir + os.sep + "cleaned_anime_data.csv")

In [2]:
df.head(3)

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,...,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched,Age
0,Damonashu,21,10,37326,Male,One Piece,TV,Manga,8.54,423868,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32
1,Damonashu,232,0,37326,Male,Cardcaptor Sakura,TV,Manga,8.21,121898,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32
2,Damonashu,249,4,37326,Male,InuYasha,TV,Manga,7.90,181978,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32


In [12]:
df["Location"].value_counts()

Location
Poland                            320457
Germany                           207170
California                        139189
France                            133671
Canada                            127451
                                   ...  
sanjose ca                             1
Asgard the land of the gods...         1
Brownsville, Texas                     1
wesleco,texas                          1
baltimore city                         1
Name: count, Length: 22546, dtype: int64

In [13]:
df_loc = df["Location"].value_counts().reset_index()
df_loc.columns = ['location', 'count']
print(df_loc.shape, type(df_loc))

(22546, 2) <class 'pandas.core.frame.DataFrame'>


In [3]:
df.shape

(14422331, 27)

In [4]:
df.columns

Index(['username', 'anime_id', 'my_score', 'user_id', 'gender', 'title',
       'type', 'source', 'score', 'scored_by', 'rank', 'popularity', 'genre',
       'Username', 'Gender', 'Location', 'Days Watched', 'Mean Score',
       'Watching', 'Completed', 'On Hold', 'Dropped', 'Plan to Watch',
       'Total Entries', 'Rewatched', 'Episodes Watched', 'Age'],
      dtype='object')

Split data into chunks with check of location validity

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm.auto import tqdm

# Enable tqdm for pandas progress_apply
tqdm.pandas()

# Initialize the geolocator with a short timeout
geolocator = Nominatim(user_agent="myGeocoder", timeout=15)

def is_location_legit(address):
    """
    Check if the given address returns a valid geocoding result.
    Returns True if a valid location is found, otherwise False.
    """
    if not address or not address.strip():
        return False
    try:
        location = geolocator.geocode(address, language="en")
        return location is not None
    except GeocoderTimedOut:
        # Optionally, implement a retry mechanism here.
        return False
    except Exception:
        return False

def process_chunk(chunk):    
    # Apply get_geo to the chunk and expand the result into new columns
    chunk['is_valid_location'] = chunk['location'].progress_apply(is_location_legit)
    # chunk.drop(columns='geo', inplace=True)
    chunk = chunk[chunk['is_valid_location']!=False]
    return chunk

chunk_out_dir = "./data/chunks"
os.makedirs(chunk_out_dir, exist_ok=True)

address_cache = {}
chunk_size = 500
chunks = []
cnt = 1
# for start in range(0, len(df_loc), chunk_size):
for start in range(0, 500, chunk_size):
    chunk = df_loc.iloc[start:start+chunk_size].copy()
    processed_chunk = process_chunk(chunk)
    print(f"Processed chunk shape = {processed_chunk.shape}")
    processed_chunk.to_csv(chunk_out_dir + os.sep +f"chunk_{cnt}.csv",index=False)
    cnt += 1

  0%|          | 0/500 [00:00<?, ?it/s]

Processed chunk shape = (492, 3)


### Extract City, State, Country values from Location

Save chunks after splitting locations into latitude, longitude, city, state, country

In [ ]:
import pandas as pd
import os
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter

# Initialize the geolocator with a longer timeout and set up rate limiting (e.g., 1 second between calls)
geolocator = Nominatim(user_agent="MyGeoLoc", timeout=15)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
reverse_geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Functions to extract city, state, and country from the address dictionary
def find_city(address):
    if 'city' in address:
        return address.get('city', '')
    elif 'city_district' in address:
        return address.get('city_district', '')
    elif 'suburb' in address:
        return address.get('suburb', '')
    elif 'town' in address:
        return address.get('town', '')
    elif 'village' in address:
        return address.get('village', '')
    return None

def find_state(address):
    if 'state' in address:
        return address.get('state', '')
    elif 'state_district' in address:
        return address.get('state_district', '')
    elif 'county' in address:
        return address.get('county', '')
    return None

def find_country(address):
    return address.get('country', None)

# Function that geocodes a given address and extracts latitude, longitude, city, state, and country
def find_location(given_address):
    lat, lon, city, state, country = None, None, None, None, None
    if given_address:
        print(given_address)
        print("="*100)
        try:
            location = geocode(given_address, language="en")
            if location:
                lat, lon = location.latitude, location.longitude
                rev_location = reverse_geocode((lat, lon),
                                               exactly_one=True,
                                               addressdetails=True,
                                               language="en")
                if rev_location:
                    addr_dict = rev_location.raw.get('address', {})
                    city = find_city(addr_dict)
                    state = find_state(addr_dict)
                    country = find_country(addr_dict)
        except Exception as e:
            # Optionally log or handle the error here
            pass
    return (lat, lon, city, state, country)


chunk_dir = "./data/chunks"
for filename in os.listdir(chunk_dir):
    print(f"Processing {filename}...")
    chunk = pd.read_csv(chunk_dir + os.sep + filename)
    if ('lat' not in chunk.columns) or ('lon' not in chunk.columns) or ('city' not in chunk.columns) or ('state' not in chunk.columns) or ('country' not in chunk.columns):
        chunk[['lat', 'lon', 'city', 'state', 'country']] = chunk['location'].progress_apply(lambda loc: pd.Series(find_location(loc)))
        chunk = chunk.dropna(subset=['lat', 'lon', 'city', 'state', 'country'])
        chunk.to_csv(chunk_dir + os.sep + filename, index=False)
    else:
        print(f"Skipping {filename}...")

Processing chunk_1.csv...
Skipping chunk_1.csv...
Processing chunk_10.csv...
Skipping chunk_10.csv...
Processing chunk_11.csv...
Skipping chunk_11.csv...
Processing chunk_12.csv...
Skipping chunk_12.csv...
Processing chunk_13.csv...
Skipping chunk_13.csv...
Processing chunk_14.csv...
Skipping chunk_14.csv...
Processing chunk_15.csv...
Skipping chunk_15.csv...
Processing chunk_16.csv...
Skipping chunk_16.csv...
Processing chunk_17.csv...
Skipping chunk_17.csv...
Processing chunk_18.csv...
Skipping chunk_18.csv...
Processing chunk_19.csv...
Skipping chunk_19.csv...
Processing chunk_2.csv...
Skipping chunk_2.csv...
Processing chunk_20.csv...
Skipping chunk_20.csv...
Processing chunk_21.csv...
Skipping chunk_21.csv...
Processing chunk_22.csv...
Skipping chunk_22.csv...
Processing chunk_23.csv...
Skipping chunk_23.csv...
Processing chunk_24.csv...
Skipping chunk_24.csv...
Processing chunk_25.csv...
Skipping chunk_25.csv...
Processing chunk_26.csv...
Skipping chunk_26.csv...
Processing chunk

  0%|          | 0/384 [00:00<?, ?it/s]

saudi arabia,Eastern Region
Araçatuba, São Paulo
RI
Salford, England
Melbourne, FL
Houston(:
Swanton, Ohio
Cergy, France
Espoo
Bulgaria , Burgas
Salt Lick, Kentucky
Birlenbach, Germany
Baton Rouge, Louisiana 533
Eiropa, Latvia
Savigny sur Orge, France
North Babylon, New York
Germany/cologne
Saginaw, Michigan
Karmøy
VB, VA
Småland,Sweden
In My Head
Poland~
Watford, UK
London, Uk
DK
haarlem, Netherlands
MNL
Africa
Lublin , Poland
Black Hills
My room =3
Sudan, Khartoum
Baguio City
Tarragona, cataluña, españa
Tbilisi;Georgia.
Erslev, Denmark
Stockport, UK
Polska, Kielce
Enschede, The Netherlands
Ukrain, Evpatoria
Qatif
Don Torcuato, Buenos Aires
England, Bristol
samarinda
Kętrzyn POLAND
ca / nj
Germany Bavaria
Halmstad, Halland
Piła, Polska
Polad, Wroclaw
La Plata, Argentina
HK
Viña del Mar, Chile.
Saguenay
Russia, Artem
Winona, Minnesota
N+C universe
Jokkmokk, Sweden
France, Côte d'Ivoire
Coral Springs, Florida
MN :3
Hamilton, Ontario, CANADA
Nurmo, Finland
Spain, Barcelona
Wakefield
Batt

  0%|          | 0/394 [00:00<?, ?it/s]

bideford UK
Jegłownik, Poland
miami florida
Gdańsk, Polska
Karvina, Czech Republic
Birkerød, Denmark
athens/greece
manila , philippines
Turnhout, Belgium
São Bernardo do Campo
Santa Ana, CA
West Jakarta, Jakarta
Austin TX
Catania,Italia
Sleaford, UK
Chisinau, Republic of Moldova
Mayo, Ireland
Mezotur, Hungary
Bergamo, Italy
newnan, ga
Milano\
Netherlands, Rotterdam
Guze
Not sure
egypt, cairo
Ustrzyki Dolne, Poland
Stargard, Poland
Kidderminster
Brasilia, DF
dayton
Russia, St Petersburg
East Greenbush, New York
NASHVILLE TENNESSEE
Snohomish, Washington
Amsterdam,The Netherlands
Kłodzko, Poland
Erfurt, Germany
Recife, Pernambuco, Brasil
Wonderland.!~
Targoviste
Siedlce, Polska
Trinoma, Philippines
Wolne Miasto Gdańsk
Valtice, Czech Republic
Buenos Aires , Argentina
MELBOURNE!!!!!!...... AUSTRALIA!!!
Ak
Syria
Aachen,NRW,Germany
Torino
Martins Ferry, Ohio
Pisa, Tuscany, Italy
Cracow,Poland
Quezon City, Manila
Aosta Valley, Italy
hermosa beach, California
Marksville, Louisiana
South Town
Ma

...

Milky Way ~
Włocławek, Polska
Kozieglowy, Poland
Kent, United Kingdom
Karlskrona, Sweden
Kent, UK
Indiana, United States
Posadas
Vero Beach, Florida
Lyon - France
Buenos Aires, Argentina.
Paraparaumu, Kapiti Coast, New Zealand
Casablanca
dawson creek, BC
Portsmouth, England
Turkey, Izmir
az-zarqa, jordan
Little Rock, Arkansas
Florence, S. Carolina
Belgrade , Serbia
Alphen aan den Rijn
Evry
Russia, Tambov
Cedar Park, Texas
South of Heaven
Maringá-PR
/home/
oregon
Montreal.Qc
Levittown, PA
Cornwall, United Kingdom
Asturias, España
Dunno
Tehran - Iran
Piemonte, Italy
Siedlce, Poland
Tomaszów Mazowiecki, Poland
Who knows D:
Bolivar, Pennsylvania
Сыктывкар
ZA WARUDO
Saitama
russia
Australia, Perth
Russia, Воронеж
Izhevsk
Morocco
NRW
Tustin, California
Bucharest RO
tangerang, banten
Brea, California
Toronto, ON
Vigo, España
In A House
Poland, Krakow
Polska, Oświęcim
Polska, Puławy
Camuy,Puerto Rico
Warsaw - Poland
Val d'Oise, France
valhalla
Aix-En-Provence
Ozark, Alabama
Eorzea
Kuźnia, Pola

  0%|          | 0/414 [00:00<?, ?it/s]

*Place Name*, New Zealand
Poznań,Polska
Corinth, Maine
Sergipe, Brazil
Slovakia, EU
Ljusdal, Sweden
Bialystok
reading, uk
Gig Harbor, Washington
Bratsk, Russia
Florida, U.S.
Rio De JAneiro
polska
Veresegyház
Sp
Resende
Cebu,Philippines
Trinidad & Tobago
Chennai, Tamil Nadu
jakarta, indonesia
Polska / Głogów
Newcastle, UK
MSK
Grotte
Bayern, Deutschland
Sydney, New South Wales, Australia
Belarus/Minsk
Belleville, Ontario
Russia, Arzamas
Kuching, Malaysia
Russia, Kazan
Ottawa, Ontario, Canada.
Belgium, Antwerp
Uppsala
Polska, Grudziądz
Holy Land
Cagayan de Oro, Philippines
Lebanon, Virginia
Ukraine, Cherkassy
Norwalk, CA
Santiago of Chile
渋谷区, 東京
United States, Texas
manila philipines
Cudahy, California
Maasmechelen, Belgium
athens
Ostrołęka, Poland
Rosemead, California
Saguenay, Québec, Canada
Tijuana Baja California
La Serena, chile
Kaposmérő, Hungary
Mohammedia
Las Piñas City, Metro Manila, Philippines
Russia Voronezh
Jackson, NJ
IN AMERICA
Canterbury,UK
New Mexico
Kudus, Indonesia
Oln

  0%|          | 0/409 [00:00<?, ?it/s]

Radebeul,Deutschland
West Michigan
Brasov / Kronstadt
Kocaeli/Turkey
Pasig, Philippines
Regensburg, Germany
Montijo
Glückstadt
Arlington, Virginia
Brno, CZ
Warminster, PA
Criciúma, Santa Catarina
~:: UAE ::~
Bangladesh
Tallinn , Estonia
stoke-on-trent
Brazil, Porto Velho
Halifax, Nova Scotia
Poznan
Albuquerque
265
Coventry, England
Актау, Казахстан
Mexico, Mexico
Limbo
In Your Dreams
In a cave
Irkutsk, Russian Federation
La Crosse, WI
Russia Omsk
Elcho, Wisconsin
Hungary, Szekesfehérvár
Köniz, Switzerland
Jerez de la Frontera
manila,philippines
Hamar, Norway
Zambrów, Poland
Area 51
Porto - Portugal
OK
Merida, Yucatan.
Marburg, Germany
Golden Spring, Jamaica
North East England
Catalunya, Spain
Washington, U.S.
Detroit,MI
belguim
Guipuzcoa
Bulgaria,Chepelare
Mandaluyong City, Philippines
LA,california
Ruskin, Florida
Crystal, Minnesota
Słupsk, Poland
Düsseldorf
Whittier, CA
Hungary, Törökszentmiklós
Mexico city, Mexico
Nice
Merry Ol' England
Death City
Nalchik, Russia
Berkshire, UK
Teren

Merge location chunks with full data

In [ ]:
out_dir = "./data/intermediate_results"
df = pd.read_csv(out_dir + os.sep + "cleaned_anime_data.csv")
df.head(3)

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,...,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched,Age
0,Damonashu,21,10,37326,Male,One Piece,TV,Manga,8.54,423868,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32
1,Damonashu,232,0,37326,Male,Cardcaptor Sakura,TV,Manga,8.21,121898,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32
2,Damonashu,249,4,37326,Male,InuYasha,TV,Manga,7.90,181978,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32


In [4]:
df.drop_duplicates().shape
#14422331

(14422331, 27)

In [5]:
df.columns

Index(['username', 'anime_id', 'my_score', 'user_id', 'gender', 'title',
       'type', 'source', 'score', 'scored_by', 'rank', 'popularity', 'genre',
       'Username', 'Gender', 'Location', 'Days Watched', 'Mean Score',
       'Watching', 'Completed', 'On Hold', 'Dropped', 'Plan to Watch',
       'Total Entries', 'Rewatched', 'Episodes Watched', 'Age'],
      dtype='object')

In [ ]:
cols = ["location","lat","lon","city","state","country"]
all_chunks = pd.DataFrame(columns=cols)
chunk_dir = chunk_out_dir = "./data/chunks"
for filename in os.listdir(chunk_dir):
    print("-"*50, filename, "-"*50)
    chunk = pd.read_csv(chunk_dir + os.sep + filename)
    print(f"Shape of chunk = {chunk.shape}")
    all_chunks = pd.concat([all_chunks, chunk[cols]], axis=0, ignore_index=True)
    print(f"Shape of ALL chunk = {all_chunks.shape}")
    # df = pd.merge(df, chunk, left_on="Location", right_on="location", how='left')
    # df.drop_duplicates(subset=cols, inplace=True)
    # print(df.shape)
    print("="*100)

    

-------------------------------------------------- chunk_1.csv --------------------------------------------------
Shape of chunk = (373, 8)
Shape of ALL chunk = (373, 6)
-------------------------------------------------- chunk_10.csv --------------------------------------------------
Shape of chunk = (353, 8)
Shape of ALL chunk = (726, 6)
-------------------------------------------------- chunk_11.csv --------------------------------------------------
Shape of chunk = (365, 8)
Shape of ALL chunk = (1091, 6)
-------------------------------------------------- chunk_12.csv --------------------------------------------------
Shape of chunk = (357, 8)
Shape of ALL chunk = (1448, 6)
-------------------------------------------------- chunk_13.csv --------------------------------------------------
Shape of chunk = (347, 8)
Shape of ALL chunk = (1795, 6)
-------------------------------------------------- chunk_14.csv --------------------------------------------------
Shape of chunk = (353, 8)
Sh

/tmp/ipykernel_508603/3193751441.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_chunks = pd.concat([all_chunks, chunk[cols]], axis=0, ignore_index=True)


Shape of chunk = (386, 8)
Shape of ALL chunk = (8007, 6)
-------------------------------------------------- chunk_30.csv --------------------------------------------------
Shape of chunk = (337, 8)
Shape of ALL chunk = (8344, 6)
-------------------------------------------------- chunk_31.csv --------------------------------------------------
Shape of chunk = (357, 8)
Shape of ALL chunk = (8701, 6)
-------------------------------------------------- chunk_32.csv --------------------------------------------------
Shape of chunk = (318, 8)
Shape of ALL chunk = (9019, 6)
-------------------------------------------------- chunk_33.csv --------------------------------------------------
Shape of chunk = (350, 8)
Shape of ALL chunk = (9369, 6)
-------------------------------------------------- chunk_34.csv --------------------------------------------------
Shape of chunk = (333, 8)
Shape of ALL chunk = (9702, 6)
-------------------------------------------------- chunk_35.csv -------------------

In [7]:
all_chunks.head(3)

,location,lat,lon,city,state,country
0,Poland,52.215933,19.134422,Bardzinek,Łódź Voivodeship,Poland
1,Germany,51.163818,10.447831,Vogtei,Thuringia,Germany
2,France,46.603354,1.888334,Sarzay,Centre-Val de Loire,France


In [8]:
df.head(3)

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,...,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched,Age
0,Damonashu,21,10,37326,Male,One Piece,TV,Manga,8.54,423868,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32
1,Damonashu,232,0,37326,Male,Cardcaptor Sakura,TV,Manga,8.21,121898,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32
2,Damonashu,249,4,37326,Male,InuYasha,TV,Manga,7.90,181978,...,6.16,6.0,216.0,34.0,37.0,29.0,322.0,6.0,5360.0,32


In [ ]:
df_cols = df.columns
df = pd.merge(df, all_chunks, left_on="Location", right_on="location", how='left')

(14422331, 33)


In [11]:
df = df.drop_duplicates(subset=df_cols)
print(df.shape)

(14422331, 33)


In [13]:
df.head(3)

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,...,Total Entries,Rewatched,Episodes Watched,Age,location,lat,lon,city,state,country
0,Damonashu,21,10,37326,Male,One Piece,TV,Manga,8.54,423868,...,322.0,6.0,5360.0,32,"Detroit,Michigan",42.331551,-83.04664,Detroit,Michigan,United States
1,Damonashu,232,0,37326,Male,Cardcaptor Sakura,TV,Manga,8.21,121898,...,322.0,6.0,5360.0,32,"Detroit,Michigan",42.331551,-83.04664,Detroit,Michigan,United States
2,Damonashu,249,4,37326,Male,InuYasha,TV,Manga,7.90,181978,...,322.0,6.0,5360.0,32,"Detroit,Michigan",42.331551,-83.04664,Detroit,Michigan,United States


In [14]:
df.columns

Index(['username', 'anime_id', 'my_score', 'user_id', 'gender', 'title',
       'type', 'source', 'score', 'scored_by', 'rank', 'popularity', 'genre',
       'Username', 'Gender', 'Location', 'Days Watched', 'Mean Score',
       'Watching', 'Completed', 'On Hold', 'Dropped', 'Plan to Watch',
       'Total Entries', 'Rewatched', 'Episodes Watched', 'Age', 'location',
       'lat', 'lon', 'city', 'state', 'country'],
      dtype='object')

In [4]:
for col in ["location","lat","lon","city","state","country"]:
    if df[col].isnull().sum()>0:
        print(f"{col} --> {df[col].isnull().sum()}")

location --> 4431816
lat --> 4431816
lon --> 4431816
city --> 4432443
state --> 4431816
country --> 4431816


In [ ]:
print(df.shape)
df = df.dropna()
print(df.shape)

(14422331, 34)
(9989888, 34)


### Filter data by country

In [4]:
df = pd.read_csv("/home/mchakravorty_umass_edu/bulky_data/dataVis_project/data/updated_anime_data.csv")
print(df.shape)

(9989888, 35)


In [5]:
df["country"].value_counts()

country
United States                       2263444
Poland                              1121580
United Kingdom                       558081
Russia                               506975
Brazil                               504588
                                     ...   
Namibia                                  43
Democratic Republic of the Congo         42
Isle of Man                              39
Lesotho                                  33
Somalia                                  21
Name: count, Length: 148, dtype: int64

In [6]:
df_usa = df[df["country"]=="United States"]
print(df_usa.shape)

(2263444, 35)


In [7]:
df_usa.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'username', 'anime_id', 'my_score',
       'user_id', 'gender', 'title', 'type', 'source', 'score', 'scored_by',
       'rank', 'popularity', 'genre', 'Username', 'Gender', 'Location',
       'Days Watched', 'Mean Score', 'Watching', 'Completed', 'On Hold',
       'Dropped', 'Plan to Watch', 'Total Entries', 'Rewatched',
       'Episodes Watched', 'Age', 'location', 'lat', 'lon', 'city', 'state',
       'country'],
      dtype='object')

In [8]:
df_usa = df_usa[['username', 'anime_id', 'my_score',
       'user_id', 'gender', 'title', 'type', 'source', 'score', 'scored_by',
       'rank', 'popularity', 'genre', 'Username', 'Gender', 'Location',
       'Days Watched', 'Mean Score', 'Watching', 'Completed', 'On Hold',
       'Dropped', 'Plan to Watch', 'Total Entries', 'Rewatched',
       'Episodes Watched', 'Age', 'location', 'lat', 'lon', 'city', 'state',
       'country']]

In [9]:
top15_states = df_usa['state'].value_counts().nlargest(15).index
df_top15 = df_usa[df_usa['state'].isin(top15_states)]


In [10]:
df_top15["state"].value_counts()

state
California        338447
New York          224596
Florida           168528
Texas             165303
Ohio              100416
Pennsylvania      100063
Washington         83289
Illinois           82048
Massachusetts      78940
New Jersey         77956
Michigan           53352
Georgia            52097
Arizona            49712
North Carolina     45927
Nevada             42587
Name: count, dtype: int64

In [ ]:
### Save first 100000 rows
# df_sample = df_top15[:100000] 
# df_sample("/home/mchakravorty_umass_edu/bulky_data/dataVis_project/data/filtered_usa_data_v2.csv", index=false)

### Randomly sample 100000 rows

In [11]:
target_size = 100000
current_size = len(df_top15)

if current_size < target_size:
    print(f"DataFrame only has {current_size} rows. Cannot sample {target_size} rows without replacement.")
    # Optionally, decide how to proceed:
    # a) Use replacement
    df_sample = df_top15.sample(n=target_size, replace=True, random_state=42).reset_index(drop=True)
    # b) Or simply use the available data without sampling
    # df_sample = df_sample.copy()
else:
    df_sample = df_top15.sample(n=target_size, random_state=42).reset_index(drop=True)

print(f"Final sample shape: {df_sample.shape}")

Final sample shape: (100000, 33)


In [12]:
df_sample.drop_duplicates().shape

(100000, 33)

### Analyze the preprocessed data

In [15]:
for state in list(set(df_sample["state"].to_list())):
    print("-"*50,state,"-"*50)
    df_state = df_sample[df_sample["state"]==state]
    display(df_state["source"].value_counts())
    print("-"*100)

-------------------------------------------------- Massachusetts --------------------------------------------------


source
Manga            2103
Original         1021
Light novel       654
Visual novel      317
Game              200
Unknown           155
Novel             150
4-koma manga       99
Other              59
Web manga          53
Book               12
Music              10
Card game           8
Digital manga       2
Picture book        2
Radio               1
Name: count, dtype: int64

----------------------------------------------------------------------------------------------------
-------------------------------------------------- New York --------------------------------------------------


source
Manga            5649
Original         2814
Light novel      1841
Visual novel      828
Game              614
Unknown           589
Novel             466
4-koma manga      356
Other             158
Web manga          97
Book               23
Card game          20
Music              18
Digital manga       5
Picture book        4
Radio               1
Name: count, dtype: int64

----------------------------------------------------------------------------------------------------
-------------------------------------------------- Washington --------------------------------------------------


...

----------------------------------------------------------------------------------------------------
-------------------------------------------------- Texas --------------------------------------------------


source
Manga            4283
Original         1957
Light novel      1470
Visual novel      631
Game              388
Unknown           376
Novel             345
4-koma manga      293
Other             142
Web manga          93
Music              19
Card game          19
Book               15
Picture book        7
Digital manga       4
Name: count, dtype: int64

----------------------------------------------------------------------------------------------------
-------------------------------------------------- Nevada --------------------------------------------------


source
Manga           1075
Original         490
Light novel      417
Visual novel     202
Game             113
Novel            106
Unknown           94
4-koma manga      76
Other             42
Web manga         28
Card game          4
Picture book       2
Radio              2
Book               1
Music              1
Name: count, dtype: int64

----------------------------------------------------------------------------------------------------


In [16]:
for state in list(set(df_sample["state"].to_list())):
    print("-"*50,state,"-"*50)
    df_state = df_sample[df_sample["state"]==state]
    display(df_state["Gender"].value_counts())
    print("-"*100)

-------------------------------------------------- Massachusetts --------------------------------------------------


Gender
Male          2602
Female        2203
Non-Binary      41
Name: count, dtype: int64

----------------------------------------------------------------------------------------------------
-------------------------------------------------- New York --------------------------------------------------


Gender
Male          9526
Female        3622
Non-Binary     335
Name: count, dtype: int64

----------------------------------------------------------------------------------------------------
-------------------------------------------------- Washington --------------------------------------------------


...

----------------------------------------------------------------------------------------------------
-------------------------------------------------- Texas --------------------------------------------------


Gender
Male          7501
Female        2494
Non-Binary      47
Name: count, dtype: int64

----------------------------------------------------------------------------------------------------
-------------------------------------------------- Nevada --------------------------------------------------


Gender
Male      2027
Female     626
Name: count, dtype: int64

----------------------------------------------------------------------------------------------------


### Save Preprocessed Data

In [18]:
df_sample.to_csv("/home/mchakravorty_umass_edu/bulky_data/dataVis_project/data/anime_random_sampled_data.csv", index=False)